In [10]:
import os
import subprocess
import pandas as pd

from global_paths import *

df = pd.read_csv(REPOS_PATH)

In [ ]:
df["contains_import"] = 0

os.makedirs(CLONE_REPOS_DIR, exist_ok=True)

for index, row in df.iterrows():
  url = row["url"]
  subprocess.run(["git", "clone", "--depth", "1", url + ".git", CLONE_REPOS_DIR + str(index)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  # print(subprocess.run(["dir", "cloned_repos"], stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout)
  found = subprocess.run(["grep", "-rE", "import datetime|import whenever|import arrow|import pendulum", CLONE_REPOS_DIR + str(index)], stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout
  df.at[index, "contains_import"] = 1 if found else 0
  print(f"{index}: {df.at[index, 'contains_import']}")
  subprocess.run(['rm', '-rf', CLONE_REPOS_DIR + str(index)])

  # FIX PATHS!
  #  
  # if index % 10 == 0 and index > 0:
  #   output_csv_path = f"{DATA_DIR}repos_with_datetime_check_{index/10}.csv"
  #   df.to_csv(output_csv_path, index=False)
  #   if (index/10 > 1):
  #     subprocess.run(['rm', f"{DATA_DIR}repos_with_datetime_check_{index/10 - 1}.csv"])

In [16]:
df = df[df["contains_import"] != 0]

output_csv_path = REPOS_WITH_DATETIME_PATH
df.to_csv(output_csv_path, index=False)